# Mapping crystalline strain
---

This notebook demonstates mapping strain in a single crystal.  It makes use of calibrations computed in calibration.ipynb.  In order, this notebook:

- load the data
- examine the data
- find the origin of the diffraction plane
- finds Bragg scattering
- finds the lattice vectors
- calculates the strain
- visualizes the strain


## Data
This notebook uses a simulated 4D-STEM dataset.  Simulations were performed by Colin Ophus, have DOI number 10.5281/zenodo.3592520, and can be [downloaded here](https://drive.google.com/file/d/1QiH7phMR0AaMkYoio3uhgTTQMOHG4l6b/view?usp=sharing).  
You should then set the `filepath_datacube` variable in the cell below.

This notebook picks up where calibration.ipynb left off, using some of the data and metadata written by calibration.ipynb to complete the generation of strain maps.  That notebook should be run before this one.  Then, update the variable `filepath_calibration` in the cell immediately following this one to point output file of calibration.ipynb.


### Version info

Last updated on 2020-04-23 with py4DSTEM version 0.12.0.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import py4DSTEM

In [ ]:
filepath_datacube = "/media/AuxDriveB/Data/4DSTEM_SampleData/py4DSTEM_sample_data/calibrationData_simulatedAuNanoplatelet_binned.h5"
filepath_calibrations = "/media/AuxDriveB/Data/4DSTEM_SampleData/py4DSTEM_sample_data/calibrationData_simulatedAuNanoplatelet_binned_processing.h5"

## Load the data

In [ ]:
# Check the file contents
py4DSTEM.io.read(filepath_datacube)
print('')
py4DSTEM.io.read(filepath_calibrations)

### What this data is

The data here is meant to represent a single 4D-STEM acquisition of some sample of interest in which the strain is to be calculated and visualized, plus a probe template for Bragg scattering detection, and metadata for calibration.

#### `datacube` (simulation_4DSTEM)

This represents the 4D-STEM scan of interest.  Here, we use simulated data of a gold nanoplatelet, in which position-depenedent strain has been introduced to the lattice.

#### `coordinates` (coordinates_calibrationdata)

The 4D `datacube` requires calibration to obtain quantitative strain maps.  Here, we use calibration data determined in the previous notebook, calibration.ipynb.

#### `probe_kernel`

To compute strain we first fit the lattice vectors, and to fit the lattice vectors we first localize the Bragg scattering.  py4DSTEM detects Bragg scattering using template matching - to use these methods, a convolution kernel based on the structure of the electron probe over vacuum is used.  Here, we use a probe kernel generated in the previous notebook, calibration.ipynb.

In [ ]:
# Load the data
datacube = py4DSTEM.io.read(filepath_datacube,data_id='simulation_4DSTEM')
coordinates = py4DSTEM.io.read(filepath_calibrations,data_id='coordinates_calibrationdata')
probe_kernel = py4DSTEM.io.read(filepath_calibrations,data_id='probe').slices['probe_kernel']

#### Prepare `coordinates`

Here we use the `coordinates` obtained in the last notebook to calibrate our data.

In [ ]:
coordinates.set_origin(0,0) # The origin obtained from the calibration dataset don't apply here!
coordinates.name = 'coordinates_strainmap'

## Inspect the data

In [ ]:
# Examine the 4D datacube
dp_max = np.max(datacube.data,axis=(0,1))
py4DSTEM.visualize.show(dp_max,scaling='log')

In [ ]:
# Get a virtual bright-field image
qx0,qy0 = 63.5,63.45
qR = 12

py4DSTEM.visualize.show(dp_max,scaling='log',
                        circle={'center':(qx0,qy0),'R':qR,'alpha':.25,'fill':True})
BF = py4DSTEM.process.virtualimage.get_virtualimage_circ(datacube,qx0,qy0,qR)
py4DSTEM.visualize.show(BF)

In [ ]:
# Examine the probe kernel
py4DSTEM.visualize.show_kernel(probe_kernel,R=30,L=64,W=3)

In [ ]:
# Check the information stored in coordinates
coordinates.show()

## Find the origin

In [ ]:
# Find the origin
qx0_meas,qy0_meas = py4DSTEM.process.calibration.get_origin(datacube=datacube)
py4DSTEM.visualize.show_image_grid(get_ar=lambda i:[qx0_meas,qy0_meas][i],H=1,W=2,cmap='RdBu')

In [ ]:
# Set a mask for outliers
mask,scores,cutoff = py4DSTEM.process.calibration.find_outlier_shifts(qx0_meas,qy0_meas,
                                                n_sigma=.5,edge_boundary=0)
#                                                n_sigma=5,edge_boundary=0)  # This less conservative mask will also work
                                                                             # The one in the line above is better!
py4DSTEM.visualize.show_hist(scores,vlines=cutoff)
py4DSTEM.visualize.show_image_grid(get_ar=lambda i:[qx0_meas,qy0_meas][i],
                                             H=1,W=2,cmap="RdBu",mask=mask==False)

In [ ]:
# Fit a plane
qx0_fit,qy0_fit,qx0_residuals,qy0_residuals = \
                    py4DSTEM.process.calibration.fit_origin(qx0_meas,qy0_meas,
                                                            mask=mask,fitfunction='parabola')
py4DSTEM.visualize.show_image_grid(lambda i:[qx0_meas,qx0_fit,qx0_residuals,
                                             qy0_meas,qy0_fit,qy0_residuals][i],
                                   H=2,W=3,cmap='RdBu')

In [ ]:
# Store the origin position
coordinates.set_origin(qx0_fit,qy0_fit)

## Find Bragg scattering

In [ ]:
# Select a few DPs on which to test disk detection parameters

rxs = 19,50,62
rys = 48,31,68
colors = ['r','b','g']

py4DSTEM.visualize.show_points(BF,x=rxs,y=rys,pointcolor=colors,figsize=(8,8))
py4DSTEM.visualize.show_image_grid(get_ar=lambda i:datacube.data[rxs[i],rys[i],:,:],
                                   H=1,W=3,get_bordercolor=lambda i:colors[i],
                                   clipvals='std',min=0,max=5)

In [ ]:
# Tune disk detection parameters on selected DPs

corrPower=1
sigma=2
edgeBoundary=10
minRelativeIntensity=0.005
relativeToPeak=0
minPeakSpacing=15
maxNumPeaks=80
subpixel='multicorr'
upsample_factor=16

selected_peaks = py4DSTEM.process.diskdetection.find_Bragg_disks_selected(
                        datacube=datacube,
                        probe=probe_kernel,
                        Rx=rxs,
                        Ry=rys,
                        corrPower=corrPower,
                        sigma=sigma,
                        edgeBoundary=edgeBoundary,
                        minRelativeIntensity=minRelativeIntensity,
                        relativeToPeak=relativeToPeak,
                        minPeakSpacing=minPeakSpacing,
                        maxNumPeaks=maxNumPeaks,
                        subpixel=subpixel,
                        upsample_factor=upsample_factor
)

py4DSTEM.visualize.show_points(BF,x=rxs,y=rys,pointcolor=colors,figsize=(8,8))
py4DSTEM.visualize.show_image_grid(get_ar=lambda i:datacube.data[rxs[i],rys[i],:,:],H=1,W=3,
                                   get_bordercolor=lambda i:colors[i],
                                   get_x=lambda i:selected_peaks[i].data['qx'],
                                   get_y=lambda i:selected_peaks[i].data['qy'],
                                   get_pointcolors=lambda i:colors[i],
                                   clipvals='std',min=0,max=5)

In [ ]:
# Get all disks

braggpeaks_raw = py4DSTEM.process.diskdetection.find_Bragg_disks(
                                datacube=datacube,
                                probe=probe_kernel,
                                corrPower=corrPower,
                                sigma=sigma,
                                edgeBoundary=edgeBoundary,
                                minRelativeIntensity=minRelativeIntensity,
                                relativeToPeak=relativeToPeak,
                                minPeakSpacing=minPeakSpacing,
                                maxNumPeaks=maxNumPeaks,
                                subpixel=subpixel,
                                upsample_factor=upsample_factor
)

In [ ]:
# Center the disk positions about the origin
braggpeaks_centered = py4DSTEM.process.calibration.center_braggpeaks(braggpeaks_raw,coords=coordinates)

In [ ]:
# Get Bragg vector map
bvm_raw = py4DSTEM.process.diskdetection.get_bragg_vector_map(
                    braggpeaks_centered,datacube.Q_Nx,datacube.Q_Ny)
py4DSTEM.visualize.show(bvm_raw,cmap='gray',scaling='log',clipvals='manual',min=0,max=12)

In [ ]:
# Get the intensities of all the Bragg disks
bragg_intensities = py4DSTEM.process.diskdetection.get_pointlistarray_intensities(braggpeaks_centered)

In [ ]:
# Set a global intensity threshold for the Bragg disks
thresh = 18
xmax = 50

fig,ax = py4DSTEM.visualize.show_hist(bragg_intensities,returnfig=True)
ax.vlines(thresh,0,ax.get_ylim()[1],ls=':',color='k')
plt.show()
fig,ax = py4DSTEM.visualize.show_hist(bragg_intensities,returnfig=True,bins=np.linspace(0,xmax,500))
ax.vlines(thresh,0,ax.get_ylim()[1],ls=':',color='k')
plt.show()

In [ ]:
# Apply the threshold
braggpeaks_thresh = py4DSTEM.process.diskdetection.universal_threshold(braggpeaks_centered,thresh,'manual')

In [ ]:
# Get and show the thresholded intensities
bragg_intensities_thresh = py4DSTEM.process.diskdetection.get_pointlistarray_intensities(braggpeaks_thresh)

fig,ax = py4DSTEM.visualize.show_hist(bragg_intensities_thresh,returnfig=True)
ax.vlines(thresh,0,ax.get_ylim()[1],ls=':',color='k')
plt.show()
fig,ax = py4DSTEM.visualize.show_hist(bragg_intensities_thresh,returnfig=True,bins=np.linspace(0,xmax,500))
ax.vlines(thresh,0,ax.get_ylim()[1],ls=':',color='k')
plt.show()

In [ ]:
# Re-compute the BVM
bvm_thresh = py4DSTEM.process.diskdetection.get_bragg_vector_map(
                    braggpeaks_thresh,datacube.Q_Nx,datacube.Q_Ny)
py4DSTEM.visualize.show(bvm_thresh,cmap='gray',scaling='log',clipvals='manual',min=0,max=12)

In [ ]:
# Correct elliptical distortion
braggpeaks_ellipsecorr = py4DSTEM.process.calibration.correct_braggpeak_elliptical_distortions(
                             braggpeaks_thresh, coordinates.get_e(),coordinates.get_theta())

In [ ]:
# Recalculate the bvm
bvm_ellipsecorr = py4DSTEM.process.diskdetection.get_bragg_vector_map(
                      braggpeaks_ellipsecorr,datacube.Q_Nx,datacube.Q_Ny)
py4DSTEM.visualize.show(bvm_ellipsecorr,cmap='gray',scaling='log',clipvals='manual',min=0,max=12)

##  Find the lattice vectors

In [ ]:
gx,gy,gI = py4DSTEM.process.utils.get_maxima_2D(
                bvm_ellipsecorr,
                sigma=0,
                edgeBoundary=10,
                minSpacing=20,
                minRelativeIntensity=0.00001,
                relativeToPeak=0,
                maxNumPeaks=80,
                subpixel=True
)
py4DSTEM.visualize.show_points(bvm_ellipsecorr,x=gx,y=gy,cmap='gray',scaling='log',clipvals='manual',min=0,max=12)

In [ ]:
# Select the center beam and two g vectors
i0,i_g1,i_g2 = 0,4,12
g1,g2 = py4DSTEM.visualize.select_lattice_vectors(bvm_ellipsecorr,scaling='log',clipvals='manual',min=0,max=12,
                                                  gx=gx,gy=gy,i0=i0,i1=i_g1,i2=i_g2)

In [ ]:
# Get indexing and show
h,k,braggdirections = py4DSTEM.process.latticevectors.index_bragg_directions(
                                        qx0,qy0,gx,gy,g1,g2)
py4DSTEM.visualize.show_bragg_indexing(bvm_ellipsecorr,scaling='log',clipvals='manual',min=0,max=12,
                                       braggdirections=braggdirections,points=True)

In [ ]:
# Select maxPeakSpacing
maxPeakSpacing = 5
py4DSTEM.visualize.show_max_peak_spacing(bvm_ellipsecorr,cmap='gray',scaling='log',clipvals='manual',min=0,max=12,
                          spacing=maxPeakSpacing,braggdirections=braggdirections)

In [ ]:
# Add indices to the braggpeaks PointListArray
braggpeaks_indexed = py4DSTEM.process.latticevectors.add_indices_to_braggpeaks(
            braggpeaks_ellipsecorr,braggdirections,maxPeakSpacing=maxPeakSpacing,qx_shift=qx0,qy_shift=qy0)

In [ ]:
# Fit reciprocal lattice vectors at each scan position
g1g2_map = py4DSTEM.process.latticevectors.fit_lattice_vectors_all_DPs(
                            braggpeaks_indexed,minNumPeaks=6)

## Calculate strain

Here we:
- find and show the strain using an automatically selected reference lattice
- select a reference lattice
- find and show the strain with this reference
- orient the strain matrix by setting a reference in diffraction space
- orient the strain matrix by setting a reference in real space

In [ ]:
# Compute the strain using the median of the measured g1g2 as reference
strainmap_mediang1g2 = py4DSTEM.process.latticevectors.get_strain_from_reference_region(
                        g1g2_map,mask=np.ones((datacube.R_Nx,datacube.R_Ny),dtype=bool))

# Show
py4DSTEM.visualize.show_strain(strainmap_mediang1g2,
            vrange_exx=[-6,6],vrange_theta=[-3,3],
            axes_x0=14,axes_y0=6,xaxis_x=1,xaxis_y=0,axes_length=10,
            axes_width=1,axes_color='w',xaxis_space='Q',
            QR_rotation=np.degrees(coordinates.get_QR_rotation()),
            axes_labelsize=16,axes_labelcolor='w',axes_plots=('exx'),
            figsize=(12,12))

In [ ]:
# Define a reference with an ROI, which is taken to have zero strain
x0,xf,y0,yf = 34,44,9,19
py4DSTEM.visualize.show(strainmap_mediang1g2.slices['e_xx'],
                        mask=strainmap_mediang1g2.slices['mask'],
                        cmap='RdBu',clipvals='centered',min=0,
                        rectangle={'lims':(x0,xf,y0,yf),'fill':False,'color':'k'})

In [ ]:
# Get reference lattice vectors
mask = np.zeros((datacube.R_Nx,datacube.R_Ny),dtype=bool)
mask[x0:xf,y0:yf] = True
g1_ref,g2_ref = py4DSTEM.process.latticevectors.get_reference_g1g2(g1g2_map,mask)
py4DSTEM.visualize.show_lattice_vectors(bvm_ellipsecorr,scaling='log',clipvals='manual',min=0,max=12,
                                        x0=datacube.Q_Ny/2.,y0=datacube.Q_Nx/2.,
                                        g1=g1,g2=g2)

In [ ]:
# Get the strain with respect to this reference lattice
strainmap_ROIg1g2 = py4DSTEM.process.latticevectors.get_strain_from_reference_g1g2(
                                g1g2_map,g1_ref,g2_ref)

# Show
py4DSTEM.visualize.show_strain(strainmap_ROIg1g2,
            vrange_exx=[-6,6],vrange_theta=[-3,3],
            axes_x0=14,axes_y0=6,xaxis_x=1,xaxis_y=0,axes_length=10,
            axes_width=1,axes_color='w',xaxis_space='Q',
            QR_rotation=np.degrees(coordinates.get_QR_rotation()),
            axes_labelsize=16,axes_labelcolor='w',axes_plots=('exx'),
            figsize=(12,12))

In [ ]:
### Orient the strain matrix

# Select an orientation for the strain matrix' x-axis
vector_space='Q'
#Qxaxis_x,Qxaxis_y = 1,0                                   # Default
Qxaxis_x,Qxaxis_y=g1[0]-g2[0],g1[1]-g2[1]

# Transform the strain
strainmap = py4DSTEM.process.latticevectors.get_rotated_strain_map(
                            strainmap_ROIg1g2,Qxaxis_x,Qxaxis_y)

# Show the strain matrix' axes
# in real and diffraction space
py4DSTEM.visualize.show_RQ_axes(
        realspace_image=strainmap.slices['e_xx'],
        realspace_pdict={'cmap':'RdBu','clipvals':'centered','min':0,
                         'mask':strainmap.slices['mask']},
        diffractionspace_image=bvm_ellipsecorr,
        diffractionspace_pdict={'scaling':'log','clipvals':'manual','min':0,'max':12},
        vx=Qxaxis_x,vy=Qxaxis_y,vector_space=vector_space,
        QR_rotation=np.degrees(coordinates.get_QR_rotation()),
        x0_R=15,y0_R=15,vlength_R=10,width_R=0.7,color_R='r',labelcolor_R='w',labelsize_R=18,
        x0_Q=62,y0_Q=62,vlength_Q=20,width_Q=1,color_Q='r',labelcolor_Q='w',labelsize_Q=18)

# Show the strain
py4DSTEM.visualize.show_strain(strainmap,vrange_exx=[-6,6],vrange_theta=[-3,3],
            axes_x0=14,axes_y0=14,xaxis_x=Qxaxis_x,xaxis_y=Qxaxis_y,axes_length=10,
            axes_width=1,axes_color='r',xaxis_space='Q',
            QR_rotation=np.degrees(coordinates.get_QR_rotation()),
            axes_labelsize=16,axes_labelcolor='w',axes_plots=('exx'),
            figsize=(12,12))

In [ ]:
# Select the orientation in real space
vector_space='R'
orientation_deg = 66
Rxaxis_x = np.cos(np.radians(orientation_deg))
Rxaxis_y = np.sin(np.radians(orientation_deg))

# Transform the strain
_,_,Qxaxis2_x,Qxaxis2_y = py4DSTEM.process.calibration.get_Qvector_from_Rvector(
            Rxaxis_x,Rxaxis_y,np.degrees(coordinates.get_QR_rotation()))
strainmap2 = py4DSTEM.process.latticevectors.get_rotated_strain_map(
            strainmap_ROIg1g2,Qxaxis2_x,Qxaxis2_y)

# Show the strain matrix' axes
# in real and diffraction space
py4DSTEM.visualize.show_RQ_axes(
        realspace_image=strainmap.slices['e_xx'],
        realspace_pdict={'cmap':'RdBu','clipvals':'centered','min':0,
                         'mask':strainmap.slices['mask']},
        diffractionspace_image=bvm_ellipsecorr,
        diffractionspace_pdict={'scaling':'log','clipvals':'manual','min':0,'max':12},
        vx=Qxaxis2_x,vy=Qxaxis2_y,vector_space=vector_space,
        QR_rotation=np.degrees(coordinates.get_QR_rotation()),
        x0_R=15,y0_R=15,vlength_R=10,width_R=0.7,color_R='r',labelcolor_R='w',labelsize_R=18,
        x0_Q=62,y0_Q=62,vlength_Q=20,width_Q=1,color_Q='r',labelcolor_Q='w',labelsize_Q=18)

# Show the strain
py4DSTEM.visualize.show_strain(strainmap2,
            vrange_exx=[-6,6],vrange_theta=[-3,3],
            axes_x0=14,axes_y0=14,
            xaxis_x=Qxaxis2_x,xaxis_y=Qxaxis2_y,axes_length=10,
            axes_width=1,axes_color='r',xaxis_space=vector_space,
            QR_rotation=np.degrees(coordinates.get_QR_rotation()),
            axes_labelsize=16,axes_labelcolor='w',axes_plots=('exx'),
            figsize=(12,12))